In [22]:
import numpy as np
import pandas as pd
np.random.seed(7)

In [2]:
def preprocessRatings(path):
    origSet = np.loadtxt(path, delimiter="::")

    users = np.unique(origSet[:, 0])
    items = np.unique(origSet[:, 1])

    trainingSet, validationSet, testSet = divideDataSet(origSet)

    trainingSet = sortDataSet(trainingSet)
    validationSet = sortDataSet(validationSet)
    testSet = sortDataSet(testSet)

    trainingRatingMatrix = getRatingMatrix(trainingSet, users, items)
    validationRatingMatrix = getRatingMatrix(validationSet, users, items)
    testRatingMatrix = getRatingMatrix(testSet, users, items)

    return trainingRatingMatrix, validationRatingMatrix, testRatingMatrix, users, items


def divideDataSet(origSet):
    np.random.shuffle(origSet)  # 打乱数据集，进行划分

    trainLength = int(0.8 * origSet.shape[0])  # 训练集 8
    validationLength = int(0.1 * origSet.shape[0])  # 验证集 1

    trainingSet = origSet[0:trainLength]  # 8
    validationSet = origSet[trainLength:trainLength + validationLength]  # 1
    testSet = origSet[trainLength + validationLength:]  # 1

    return trainingSet, validationSet, testSet


def sortDataSet(dataSet):  # 时间戳从小到大排序
    return dataSet[np.argsort(dataSet[:, 3])]


def getRatingMatrix(dataset, users, items):
    ratingMatrixDF = pd.DataFrame(index=users, columns=items)
    for i in range(dataset.shape[0]):
        ratingMatrixDF.at[dataset[i][0], dataset[i][1]] = dataset[i][2]
    ratingMatrixDF = ratingMatrixDF.fillna(0)
    return ratingMatrixDF.values


In [3]:

def train(trainingRatingMatrix, validationRatingMatrix, K, alpha, beta, epochs):
    """
    :param trainingRatingMatrix:训练集评分矩阵
    :param validationRatingMatrix:验证集评分矩阵
    :param K:维度
    :param alpha:学习率
    :param beta:惩罚系数
    :param epochs:迭代上限
    """
    userMatrix, itemMatrix = matrixFactorize(trainingRatingMatrix,
                                             K=K,
                                             alpha=alpha,
                                             beta=beta,
                                             epochs=epochs
                                             )

    predictedRatingMatrix = predict(userMatrix, itemMatrix)

    mseValidation = evaluate(predictedRatingMatrix, validationRatingMatrix)

    return {"userMatrix": userMatrix,
            "itemMatrix": itemMatrix,
            "predictedRatingMatrix": predictedRatingMatrix,
            "mseValidation": mseValidation,
            "K": K,
            "alpha": alpha,
            "beta": beta,
            "epochs": epochs}


def matrixFactorize(origMatrix, K=3, alpha=0.01, beta=0.01, epochs=100000):
    """
    :param origMatrix:待分解矩阵
    :param K:维度
    :param alpha:学习率
    :param beta:惩罚系数
    :param epochs:迭代上限
    """
    # 初始化
    M = origMatrix.shape[0]  # 矩阵大小M*N
    N = origMatrix.shape[1]
    U = np.random.random((M, K))  # 用户矩阵大小M*K
    V = np.random.random((K, N))  # 物品矩阵大小K*N
    loss = 0.  # 损失
    nonzeroNumbers = origMatrix[origMatrix > 0].shape[0]  # 矩阵中有效值数目
    index = origMatrix.nonzero()

    # FunkSVD
    # loss(u_ik,v_kj)=(r_ij-sum(k=1->K, u_ik*v_kj))**2 +
    #                   beta/2*sum(l=1->K, u_ik**2+v_kj**2)
    # 随机梯度下降求解，令e=r_ij-sum(l=1->K, u_ik*v_kj)=r_ij-rHat_ij
    # Gu_loss=-2e_ij*v_kj+beta*u_ik
    # Gv_loss=-2e_ij*u_ik+beta*v_kj
    # 迭代u_il-=Gu_loss, v_lj-=Gv_loss
    for epoch in range(epochs):
        ndx = np.random.choice(nonzeroNumbers)
        # 更新 U,V矩阵
        i = index[0][ndx]
        j = index[1][ndx]
        r = origMatrix[i, j]
        Ui = U[i, :]
        Vj = V[:, j]
        rHat = np.sum(np.dot(Ui, Vj))  # 矩阵化计算
        e = r - rHat
        for k in range(K):  # 更新一行/一列
            Gu = beta * U[i, k] - 2 * e * V[k, j]
            Gv = beta * V[k, j] - 2 * e * U[i, k]
            U[i, k] -= alpha * Gu
            V[k, j] -= alpha * Gv

        # 计算损失
        loss = 0.
        reg = np.sum(Ui ** 2) + np.sum(Vj ** 2)
        e = r - rHat
        loss += e ** 2 + beta * reg / 2

        if epoch % int((epochs / 10)) == 0:
            print("loss={} at epoch {}".format(loss, epoch))
        epoch += 1
    print("loss={} at epoch {}".format(loss, epochs))
    return U, V


def predict(userMatrix, itemMatrix):
    return np.dot(userMatrix, itemMatrix)


In [42]:
def addResultToHistory(history, result):
    history.append(result)
    updateHistory(history)


def updateHistory(history):
    if len(history) > 2:
        newMseValidation = history[len(history) - 1]["mseValidation"]
        if newMseValidation < history[history[0]]["mseValidation"]:
            history[0] = len(history) - 1
    else:
        history[0] = 1


def printBestHistory(history):
    print("\n**BEST: K={}, alpha={}, beta={}, epochs={}"
          .format(history[history[0]]["K"],
                  history[history[0]]["alpha"],
                  history[history[0]]["beta"],
                  history[history[0]]["epochs"]))


In [43]:
def evaluate(predictedRatingMatrix, testRatingMatrix):
    testValues = testRatingMatrix[testRatingMatrix > 0]

    tempMatrix = testRatingMatrix.copy()
    tempMatrix[tempMatrix == 0] = -999
    tempMatrix[tempMatrix > 0] = 0

    compareRatingMatrix = predictedRatingMatrix + tempMatrix
    compareValues = compareRatingMatrix[compareRatingMatrix >= 0]

    mse = np.sum((testValues - compareValues) ** 2) / testValues.shape[0]
    print("MSE={}".format(mse))
    return mse


def evaluateBestFromHistory(history, testRatingMatrix):
    return evaluate(history[history[0]]["predictedRatingMatrix"], testRatingMatrix)


In [7]:
trainingRatingMatrix, validationRatingMatrix, testRatingMatrix, users, items = \
        preprocessRatings("./data/ml-1m/ratings.dat")

In [44]:
history = [-1]  # 保存历史记录，首项表示最佳结果在列表中的位置

In [45]:
# 试运行
addResultToHistory(history, train(trainingRatingMatrix, validationRatingMatrix,
                                      K=3,
                                      alpha=0.01,
                                      beta=0.1,
                                      epochs=10000))
printBestHistory(history)

loss=17.222756879480823 at epoch 0
loss=9.121706310166145 at epoch 1000
loss=3.331996322384145 at epoch 2000
loss=9.235022816296775 at epoch 3000
loss=5.125688357567351 at epoch 4000
loss=10.408921792125573 at epoch 5000
loss=13.151825395447638 at epoch 6000
loss=12.160840694757791 at epoch 7000
loss=11.738952885836724 at epoch 8000
loss=4.7445752748265555 at epoch 9000
loss=0.10037310526502088 at epoch 10000
MSE=6.58847978735136

**BEST: K=3, alpha=0.01, beta=0.1, epochs=10000


In [46]:
# 简化为函数
def tuning(K, alpha, beta, epochs):
    addResultToHistory(history, train(trainingRatingMatrix, validationRatingMatrix,
                                      K=K,
                                      alpha=alpha,
                                      beta=beta,
                                      epochs=epochs))
    printBestHistory(history)

In [47]:
tuning(K=3, alpha=0.01, beta=0.2, epochs=10000)

loss=3.9263979333244343 at epoch 0
loss=1.401836491986309 at epoch 1000
loss=2.172307051610692 at epoch 2000
loss=15.230588938639649 at epoch 3000
loss=20.110090907553182 at epoch 4000
loss=8.133308313450149 at epoch 5000
loss=6.108563266562525 at epoch 6000
loss=1.1010153162544511 at epoch 7000
loss=6.218558226003691 at epoch 8000
loss=4.363911276157518 at epoch 9000
loss=16.699035094965705 at epoch 10000
MSE=6.684994121102646

**BEST: K=3, alpha=0.01, beta=0.1, epochs=10000


In [48]:
tuning(K=3, alpha=0.01, beta=0.1, epochs=100000)

loss=20.65370930077746 at epoch 0
loss=13.557907808948805 at epoch 10000
loss=1.9200931722382037 at epoch 20000
loss=0.5523371766098277 at epoch 30000
loss=0.36144479847665495 at epoch 40000
loss=0.6181662496180376 at epoch 50000
loss=1.3967300146979726 at epoch 60000
loss=0.4666551590179365 at epoch 70000
loss=0.6841616328312088 at epoch 80000
loss=0.3380949856912189 at epoch 90000
loss=0.3882746082030767 at epoch 100000
MSE=1.4637237650472976

**BEST: K=3, alpha=0.01, beta=0.1, epochs=100000


In [49]:
tuning(K=3, alpha=0.01, beta=0.1, epochs=1000000)

loss=14.939408789336378 at epoch 0
loss=0.5435441687962376 at epoch 100000
loss=0.9229520181251438 at epoch 200000
loss=0.5157615612047133 at epoch 300000
loss=0.9219926806683283 at epoch 400000
loss=0.908467280863013 at epoch 500000
loss=0.6323435445603771 at epoch 600000
loss=0.41767395688197073 at epoch 700000
loss=0.3028995105637922 at epoch 800000
loss=1.2555465212894383 at epoch 900000
loss=0.5473301612767625 at epoch 1000000
MSE=0.9106088086635298

**BEST: K=3, alpha=0.01, beta=0.1, epochs=1000000


In [50]:
tuning(K=3, alpha=0.02, beta=0.1, epochs=1000000)

loss=16.292658771757015 at epoch 0
loss=0.3400119312549515 at epoch 100000
loss=0.5725272016779401 at epoch 200000
loss=0.3233054809508321 at epoch 300000
loss=4.475839378591118 at epoch 400000
loss=0.8651430266129487 at epoch 500000
loss=2.9065090626654904 at epoch 600000
loss=0.8125081511215382 at epoch 700000
loss=0.49162732783516383 at epoch 800000
loss=0.5526485271374608 at epoch 900000
loss=0.9101016282485962 at epoch 1000000
MSE=0.9734410116847462

**BEST: K=3, alpha=0.01, beta=0.1, epochs=1000000


In [52]:
tuning(K=3, alpha=0.005, beta=0.1, epochs=1000000)

loss=5.069006559933601 at epoch 0
loss=1.7134336410148896 at epoch 100000
loss=2.0371705986149578 at epoch 200000
loss=0.3879001415008183 at epoch 300000
loss=0.301947694113473 at epoch 400000
loss=1.397141256265204 at epoch 500000
loss=0.6469373612927374 at epoch 600000
loss=0.5914790543965728 at epoch 700000
loss=0.7052344168554003 at epoch 800000
loss=2.0542142445779676 at epoch 900000
loss=0.6182050057750541 at epoch 1000000
MSE=0.9102662435299986

**BEST: K=3, alpha=0.005, beta=0.1, epochs=1000000


In [53]:
tuning(K=3, alpha=0.005, beta=0.15, epochs=1000000) 

loss=0.42792786136529704 at epoch 0
loss=1.9927681235110117 at epoch 100000
loss=0.7004313964409024 at epoch 200000
loss=1.4352898998728103 at epoch 300000
loss=0.9305338637462103 at epoch 400000
loss=3.3119819427208452 at epoch 500000
loss=2.3350601285801176 at epoch 600000
loss=2.039166686977345 at epoch 700000
loss=0.8219637071807313 at epoch 800000
loss=3.0776875673952597 at epoch 900000
loss=0.32224528436932476 at epoch 1000000
MSE=0.9122028982794002

**BEST: K=3, alpha=0.005, beta=0.1, epochs=1000000


In [54]:
tuning(K=5, alpha=0.005, beta=0.1, epochs=1000000)

loss=9.180761070016406 at epoch 0
loss=0.5150896833864249 at epoch 100000
loss=4.196575653647486 at epoch 200000
loss=5.455283983864686 at epoch 300000
loss=6.8015201233578235 at epoch 400000
loss=2.2650003081443724 at epoch 500000
loss=0.5254069474458757 at epoch 600000
loss=0.5438388349159091 at epoch 700000
loss=0.6867186214213847 at epoch 800000
loss=2.788346477514761 at epoch 900000
loss=0.5542645095605145 at epoch 1000000
MSE=0.8964676222741291

**BEST: K=5, alpha=0.005, beta=0.1, epochs=1000000


In [55]:
tuning(K=10, alpha=0.005, beta=0.1, epochs=1000000)

loss=2.996787313033341 at epoch 0
loss=0.45576305958342217 at epoch 100000
loss=2.9914514753757695 at epoch 200000
loss=0.35704988655731734 at epoch 300000
loss=0.9545012383378398 at epoch 400000
loss=1.1766381462545596 at epoch 500000
loss=0.7975486186737125 at epoch 600000
loss=0.6445308000848589 at epoch 700000
loss=0.6123346618933279 at epoch 800000
loss=1.6682663595215232 at epoch 900000
loss=0.2994228500598527 at epoch 1000000
MSE=0.8833474701492304

**BEST: K=10, alpha=0.005, beta=0.1, epochs=1000000


In [57]:
tuning(K=15, alpha=0.005, beta=0.1, epochs=1000000)

loss=0.38896469670091344 at epoch 0
loss=0.7230378235910954 at epoch 100000
loss=6.703642783314979 at epoch 200000
loss=0.8247628323521079 at epoch 300000
loss=1.150052774110234 at epoch 400000
loss=0.43304960022258887 at epoch 500000
loss=1.067655975146721 at epoch 600000
loss=0.5707358078545771 at epoch 700000
loss=0.6659576889383774 at epoch 800000
loss=0.428225254929078 at epoch 900000
loss=0.4685736832640717 at epoch 1000000
MSE=0.8875405184601376

**BEST: K=10, alpha=0.005, beta=0.1, epochs=1000000


In [62]:
evaluateBestFromHistory(history, testRatingMatrix)

MSE=0.887082109099703


0.887082109099703